In [19]:
##problem 14

import numpy as np
import sys

class DTree:
    def __init__(self, theta, feature, value = None):
        self.theta = theta
        self.feature = feature
        self.value = value
        self.left = None
        self.right = None

def gini(y):
    if len(y) == 0:
        return 1
    
    mu = np.sum(y == 1)
    mu = mu/len(y)
    g = 1 - mu**2 - (1-mu)**2
    return g

def decision_stump(X, y):
    _feature = 0
    _theta = 0
    _min_err = float('inf')
    
    for i in range(X.shape[1]):
        ## i column of data x
        x = X[:, i]
        
        ##generator theta
        x_sort = np.sort(x)
        
        theta = []
        for j in range(len(x_sort) - 1):
            theta.append((x_sort[j+1] + x_sort[j])/2 )
        
        theta.insert(0, (x_sort[0]-1))
        theta.append((x_sort[-1]+1))
        
        ##calculate err
        for j in range(len(theta)):
            ##divide y into y1 and y2, y1 for x < theta, y2 for x >= theta
            y1 = y[x < theta[j]]
            y2 = y[x >= theta[j]]
            
            ##calculate err based on Gini idx
            gi1 = gini(y1)
            gi2 = gini(y2)
            err = len(y1)*gi1 + len(y2)*gi2

            if _min_err > err: 
                _min_err = err
                _theta = theta[j]
                _feature = i

    return _feature, _theta, _min_err

def stop(X, y):
    ## if all y the same
    n1 = np.sum(y != y[0])
    
    ## if all x the same
    n2 = np.sum(X != X[0, :])
    
    return n1 == 0 or n2 == 0

def DecisionTree(X,y):
    if stop(X, y):
        #print("stop! ")
        g = DTree(None, None, y[0])
        return g
    
    else:
        ##learning criteria based on decision stump
        feature, theta, score = decision_stump(X, y)
        g = DTree(theta, feature, None)
        #print(feature, theta, score)
        ##plit data in to two parts
        x1 = X[X[:, feature] < theta]
        y1 = y[X[:, feature] < theta]
        #print(x1.shape)
        x2 = X[X[:, feature] >= theta]
        y2 = y[X[:, feature] >= theta]
        #print(x2.shape)
        
        ##build sub_tree 
        g1 = DecisionTree(x1, y1)
        g2 = DecisionTree(x2, y2)
        
        g.left = g1
        g.right = g2
        
        ##return tree data
        return g

def predict_helper(tree, X):
    if tree.value != None:
        return tree.value
    
    if X[tree.feature] < tree.theta:
        return predict_helper(tree.left, X)
    else:
        return predict_helper(tree.right, X)
    
def predict(tree, X):
    y = [predict_helper(tree, x) for x in X ]
    return y

def score(tree, pred_y, y):
    E_out = 0
    for i in range(y_out.shape[0]):
        if(pred_y[i] != y[i]):
            E_out += 1
    E_out /= y.shape[0]
    return E_out
    

    
#main
data_in = np.genfromtxt("hw6_train.dat.txt")
data_out = np.genfromtxt("hw6_test.dat.txt")
X = data_in[:, :-1]
y = data_in[:, -1]

X_out = data_out[:, :-1]
y_out = data_out[:, -1]

##learn tree
G = DecisionTree(X, y)

##predict output data
pred_y = predict(G, X_out)

##calculate E_out
E_out = score(G, pred_y, y_out)
print(E_out)

0.166


In [89]:
##problem 15
import random
def bagging(data):
    selected_idx = []
    idx = random.randint(0, data.shape[0]-1)
    selected_idx.append(idx)
    
    new_data = data[idx, :]
    new_data = new_data.reshape((1, -1))
    for i in range(500-1):
        idx = random.randint(0, data.shape[0]-1)
        selected_idx.append(idx)
        row = data[idx, :]
        row = row.reshape((1,-1))
        new_data = np.concatenate((new_data,row), axis=0)
    
    split_train_x = new_data[:, :-1]
    split_train_y = new_data[:, -1]
    
    return split_train_x, split_train_y, selected_idx

def random_forest(data_in, train_X, train_y, test_X, test_y, num):
    tmp_data = data_in
    #train_X = data_in[:, :-1]
    #train_y = data_in[:, -1]

    avg_Eout = 0
    G_in= np.zeros((train_y.shape[0],1))
    G_in = G_in.squeeze()
    G_out= np.zeros((test_y.shape[0],1))
    G_out = G_out.squeeze()
    G_oob= np.zeros((test_y.shape[0],1))
    G_oob = G_oob.squeeze()
    
    for i in range(num):
        print(i)  
        split_train_X, split_train_y, selected_idx= bagging(data_in)
        G = DecisionTree(split_train_X, split_train_y)
        
        #print(train_y)
        
        ##for E_in
        pred_train_y = predict(G, train_X)
        ##for E_out
        pred_test_y = predict(G, test_X)
        ##for average E_out
        E_out = score(G, pred_test_y, test_y)
        ##for E_oob
        selected_idx.sort()
        #print(selected_idx)
        #print(0 in selected_idx)
        for i in range(data_in.shape[0]):
            if i not in selected_idx:
                G_oob[i] += predict_helper(G, train_X[i, :])
        
        avg_Eout += E_out
        G_in += pred_train_y
        G_out += pred_test_y
    
    avg_Eout /= num
    return avg_Eout, G_in, G_out, G_oob

#main
data_in = np.genfromtxt("hw6_train.dat.txt")
data_out = np.genfromtxt("hw6_test.dat.txt")
X = data_in[:, :-1]
y = data_in[:, -1]

X_out = data_out[:, :-1]
y_out = data_out[:, -1]

avg_Eout, G_in, G_out, G_oob = random_forest(data_in, X, y, X_out, y_out, 2000)

g_in = np.sign(G_in)
for i in range(g_in.shape[0]):
    if(g_in[i] == 0):
        g_in[i] = -1
g_out = np.sign(G_out)
for i in range(g_out.shape[0]):
    if(g_out[i] == 0):
        g_out[i] = -1
        
g_oob = np.sign(G_oob)
for i in range(g_oob.shape[0]):
    if(g_oob[i] == 0):
        g_oob[i] = -1   
        
E_in = 0
for i in range(y.shape[0]):
    if(g_in[i] != y[i]):
        E_in += 1
E_in /= y.shape[0]

E_out = 0
for i in range(y_out.shape[0]):
    if(g_out[i] != y_out[i]):
        E_out += 1
E_out /= y_out.shape[0]

E_oob = 0
for i in range(y.shape[0]):
    if(g_oob[i] != y[i]):
        E_oob += 1
E_oob /= y.shape[0]

print(avg_Eout)
print(E_in)
print(E_out)
print(E_oob)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.23115000000000005
0.014
0.155
0.079


In [85]:
print(0 in selected_idx)

NameError: name 'selected_idx' is not defined